# SVR Forecasting Model

In [1]:
import tseriesRoutines as routines
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import sqlite3
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.externals import joblib

In [2]:
# RESULT REPRODUCIBILITY
# The below is necessary for starting Numpy generated random numbers
# in a well-defined initial state.
np.random.seed(42)

In [3]:
def genData(mongoid, conn, cursor, impute=True, freq='daily'):
    '''
    Generate a timeseries dataframe for timeseries modelling.
    mongoid: str. string of mongodb id.
    conn: sqlite3 connection.
    cursor: sqlite3 cursor.
    impute:
    freq:
    actualrevcount:
    '''
    np.random.seed(42)
    initial = routines.sqlToDf(conn, cursor)
    allproduct = initial.selectReview3(mongoid, impute=impute)
    product = routines.tsSalesRateSentiment(allproduct, freq=freq)
    return product
    # product = genData('5aa2ad7735d6d34b0032a795', conn, c, impute=True, 
    #   freq='daily')

In [4]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    # https://machinelearningmastery.com/multivariate-time-series-forecasting-lstms-keras/
    n_vars = 1 if type(data) is list else data.shape[1]
    df = data.copy()
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]

    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]

    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg = agg.dropna()
    return agg

In [5]:
#Adj R square
def adj_r2_score(model,X_test, y_test,):
    y_pred = model.predict(X_test)
    # model.coefs_ doesn't exist
    adj = 1 - float(len(y_test)-1)/(len(y_test)-model.n_features_-1) * \
            (1 - r2_score(y_test,y_pred))
    return adj

# Evalute random search
def evaluate(model, X_test, y_test):
    y_pred = model.predict(X_test)
    MSE = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
#    r2_adjusted = adj_r2_score(model, X_test, y_test)
    print('Model Validation')
    print('MSE: {0}'.format(MSE))
    print('R^2: {0}'.format(r2))
#    print('R^2 Adjusted: {0}'.format(r2_adjusted))

In [6]:
def splitDataNN(df, n_in=1, n_out=1, scale=True, percent=0.2):
    '''
    df: pandas dataframe. 3 columns (sales, rating, ovsentiment) with date as index
    n_in:
    n_out:
    scale:
    percent:
    X_train, y_train, X_test, y_test, dftrain = splitDataNN(product, n_in=1, 
        n_out=1, scale=True, percent=0.2)
    '''
    dftrain = series_to_supervised(df, n_in=n_in, n_out=n_out)
    # specific to this case
    dftrain = dftrain.drop(dftrain.columns[[4, 5]], axis=1)
    dftrain = dftrain[['var1(t-1)', 'var1(t)']]
    values = dftrain.values

    if scale:
        scaler = StandardScaler()
        values = scaler.fit_transform(values)
    else:
        pass

    # training data
    X, y = values[:, :-1], values[:, -1]
    # train and test set
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=percent, 
            shuffle=False, random_state=42)
    return X_train, y_train, X_test, y_test, dftrain, scaler

In [7]:
def evalForecast(model, X, y, inverse=False, scaler=None):
    '''
    Evaluate time series forecasting model
    '''
    if inverse and scaler:
        # make prediction
        ypred = model.predict(X)
        # invert scaling predicted data
        inv_ypred = np.concatenate((X, ypred.reshape((-1,1))), axis=1)
        inv_ypred = scaler.inverse_transform(inv_ypred)
        inv_ypred = inv_ypred[:, -1]
        # invert scaling for actual data
        inv_y = np.concatenate((X, y.reshape((-1,1))), axis=1)
        inv_y = scaler.inverse_transform(inv_y)
        inv_y = inv_y[:, -1]
        # RMSE
        rmse = np.sqrt(mean_squared_error(y_pred=inv_ypred, y_true=inv_y))
        # MAE
        mae = mean_absolute_error(y_pred=inv_ypred, y_true=inv_y)
    else:
        # make prediction
        ypred = model.predict(X)
        # RMSE
        rmse = np.sqrt(mean_squared_error(y_pred=ypred, y_true=y))
        # MAE
        mae = mean_absolute_error(y_pred=ypred, y_true=y)

    print('Validasi RMSE: {0:.5f}'.format(rmse))
    print('Validasi MAE: {0:.5f}'.format(mae))

In [8]:
# make connection to sqlite db
conn = sqlite3.connect('product.db')
c = conn.cursor()

# enable foreign keys
c.execute("PRAGMA foreign_keys = ON")
conn.commit()

<br>pilihan:
>     2 data di database product.db dgn review > 900:
>         5aa2ad7735d6d34b0032a795
>         5aa39533ae1f941be7165ecd
>     cluster 3
>         5a93e8768cbad97881597597
>         or 
>         5a95d7ae35d6d33d3fea56ff
>     cluster 1
>         5aa2c35e35d6d34b0032a796
>     cluster 2 
>         5a92474635d6d32207bcd343
</br>

## <font color=blue> 1. Mongodb ID: 5aa2ad7735d6d34b0032a795 </font>

In [9]:
product = genData('5aa2ad7735d6d34b0032a795', conn, c, impute=True, freq='daily')
X_train, y_train, X_test, y_test, dftrain, scaler = splitDataNN(product, percent=0.2)

In [10]:
dftrain.head()

,var1(t-1),var1(t)
date,,
2016-11-07,82.0,88
2016-11-08,88.0,74
2016-11-09,74.0,72
2016-11-10,72.0,98
2016-11-11,98.0,89


In [11]:
product['Sales'].sum()

41548

In [12]:
product.head()

,Sales,rating,ovsentiment
date,,,
2016-11-06,82,3.500000,0.500000
2016-11-07,88,5.000000,1.000000
2016-11-08,74,0.000000,0.000000
2016-11-09,72,0.000000,0.000000
2016-11-10,98,4.666667,0.333333


In [13]:
X_train.shape

(382, 1)

In [11]:
from sklearn.svm import SVR
regressor = SVR(kernel='poly')

In [15]:
# Grid Search
from sklearn.model_selection import GridSearchCV
parameters = [{'C':[1, 5, 10, 100], 'kernel':['linear']},
        {'C':[1, 5, 10, 100], 'kernel':['rbf'], 'gamma':[0.5, 0.1, 0.01, 0.001, 0.0001]}
        ]
grid_search = GridSearchCV(estimator=regressor,
        param_grid=parameters,
        scoring=['neg_mean_squared_error', 'r2', 'explained_variance'],
        cv=5,
        n_jobs=-1,
        refit='r2',
        verbose=2)

In [16]:
grid_search = grid_search.fit(X_train, y_train)
best_accuracy = grid_search.best_score_
best_parameters = grid_search.best_params_
best_estimator = grid_search.best_estimator_

Fitting 5 folds for each of 24 candidates, totalling 120 fits
[CV] C=1, kernel=linear ..............................................
[CV] C=1, kernel=linear ..............................................
[CV] C=1, kernel=linear ..............................................
[CV] C=5, kernel=linear ..............................................
[CV] C=1, kernel=linear ..............................................
[CV] C=1, kernel=linear ..............................................
[CV] C=5, kernel=linear ..............................................
[CV] C=5, kernel=linear ..............................................
[CV] C=5, kernel=linear ..............................................
[CV] C=10, kernel=linear .............................................
[CV] C=5, kernel=linear ..............................................
[CV] ............................... C=1, kernel=linear, total=   0.0s
[CV] ............................... C=1, kernel=linear, total=   0.0s
[CV] ..........

[CV] ...................... C=5, gamma=0.01, kernel=rbf, total=   0.0s
[CV] ..................... C=100, gamma=0.5, kernel=rbf, total=   0.0s
[CV] C=5, gamma=0.01, kernel=rbf .....................................
[CV] .................... C=10, gamma=0.001, kernel=rbf, total=   0.0s
[CV] C=100, gamma=0.5, kernel=rbf ....................................
[CV] C=10, gamma=0.001, kernel=rbf ...................................
[CV] ................... C=100, gamma=0.001, kernel=rbf, total=   0.0s
[CV] C=100, gamma=0.001, kernel=rbf ..................................
[CV] ....................... C=1, gamma=0.1, kernel=rbf, total=   0.0s
[CV] C=1, gamma=0.1, kernel=rbf ......................................
[CV] ...................... C=10, gamma=0.5, kernel=rbf, total=   0.0s
[CV] .................... C=1, gamma=0.0001, kernel=rbf, total=   0.0s
[CV] C=1, gamma=0.0001, kernel=rbf ...................................
[CV] C=10, gamma=0.5, kernel=rbf .....................................
[CV] .

[CV] ...................... C=10, gamma=0.1, kernel=rbf, total=   0.0s
[CV] C=100, gamma=0.01, kernel=rbf ...................................
[CV] C=10, gamma=0.1, kernel=rbf .....................................
[CV] .................... C=100, gamma=0.01, kernel=rbf, total=   0.0s
[CV] C=100, gamma=0.01, kernel=rbf ...................................
[CV] ...................... C=10, gamma=0.1, kernel=rbf, total=   0.0s
[CV] .................... C=100, gamma=0.01, kernel=rbf, total=   0.0s
[CV] C=100, gamma=0.01, kernel=rbf ...................................
[CV] .................... C=100, gamma=0.01, kernel=rbf, total=   0.0s


[Parallel(n_jobs=-1)]: Done  36 out of 120 | elapsed:    0.3s remaining:    0.7s
[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:    0.4s finished


In [17]:
best_parameters

{'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}

In [18]:
# k-fold cross validation
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator=best_estimator, X=X_train, y=y_train, cv=10, n_jobs=-1,
        scoring='neg_mean_squared_error', verbose=2)
accuracies.mean(), accuracies.std()

[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV] ................................................. , total=   0.0s
[CV]  ................................................................
[CV] ................................................. , total=   0.0s
[CV] ................................................. , total=   0.0s
[CV] ................................................. , total=   0.0s
[CV] .

[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.0s finished


(-0.9979816172941856, 0.2873551907974374)

In [19]:
evalForecast(best_estimator, X_train, y_train, inverse=True, scaler=scaler)

Validasi RMSE: 9.28719
Validasi MAE: 7.50273


In [22]:
#joblib.dump(best_estimator, './training/forecast_svr_5aa2ad7735d6d34b0032a795_UI.pkl')

## <font color=blue> 2. Mongodb ID: 5aa39533ae1f941be7165ecd </font>

In [9]:
product = genData('5aa39533ae1f941be7165ecd', conn, c, impute=True, freq='daily')
X_train, y_train, X_test, y_test, dftrain, scaler = splitDataNN(product, percent=0.2)

In [12]:
regressor = SVR(kernel='poly')

In [13]:
# Grid Search
from sklearn.model_selection import GridSearchCV
parameters = [{'C':[1, 5, 10, 100], 'kernel':['linear']},
        {'C':[1, 5, 10, 100], 'kernel':['rbf'], 'gamma':[0.5, 0.1, 0.01, 0.001, 0.0001]}
        ]
grid_search = GridSearchCV(estimator=regressor,
        param_grid=parameters,
        scoring=['neg_mean_squared_error', 'r2', 'explained_variance'],
        cv=5,
        n_jobs=-1,
        refit='r2',
        verbose=2)

In [14]:
grid_search = grid_search.fit(X_train, y_train)
best_accuracy = grid_search.best_score_
best_parameters = grid_search.best_params_
best_estimator = grid_search.best_estimator_

Fitting 5 folds for each of 24 candidates, totalling 120 fits
[CV] C=1, kernel=linear ..............................................
[CV] C=1, kernel=linear ..............................................
[CV] C=1, kernel=linear ..............................................
[CV] C=1, kernel=linear ..............................................
[CV] ............................... C=1, kernel=linear, total=   0.0s
[CV] ............................... C=1, kernel=linear, total=   0.0s
[CV] C=1, kernel=linear ..............................................
[CV] C=5, kernel=linear ..............................................
[CV] ............................... C=1, kernel=linear, total=   0.0s
[CV] C=5, kernel=linear ..............................................
[CV] ............................... C=1, kernel=linear, total=   0.0s
[CV] C=5, kernel=linear ..............................................
[CV] ............................... C=1, kernel=linear, total=   0.0s
[CV] ..........

[CV] C=5, gamma=0.01, kernel=rbf .....................................
[CV] ............................. C=100, kernel=linear, total=   0.1s
[CV] C=100, kernel=linear ............................................
[CV] ..................... C=10, gamma=0.01, kernel=rbf, total=   0.0s
[CV] C=10, gamma=0.01, kernel=rbf ....................................
[CV] .................... C=5, gamma=0.0001, kernel=rbf, total=   0.0s
[CV] ...................... C=5, gamma=0.01, kernel=rbf, total=   0.0s
[CV] C=5, gamma=0.01, kernel=rbf .....................................
[CV] C=10, gamma=0.5, kernel=rbf .....................................
[CV] ..................... C=10, gamma=0.01, kernel=rbf, total=   0.0s
[CV] C=10, gamma=0.01, kernel=rbf ....................................
[CV] ............................. C=100, kernel=linear, total=   0.1s
[CV] C=10, gamma=0.0001, kernel=rbf ..................................
[CV] ...................... C=10, gamma=0.5, kernel=rbf, total=   0.0s
[CV] .

[CV] ................... C=100, gamma=0.001, kernel=rbf, total=   0.0s
[CV] C=100, gamma=0.001, kernel=rbf ..................................
[CV] ................... C=100, gamma=0.001, kernel=rbf, total=   0.0s
[CV] C=100, gamma=0.001, kernel=rbf ..................................
[CV] ................... C=100, gamma=0.001, kernel=rbf, total=   0.0s
[CV] C=100, gamma=0.001, kernel=rbf ..................................
[CV] ................... C=100, gamma=0.001, kernel=rbf, total=   0.0s
[CV] C=100, gamma=0.0001, kernel=rbf .................................
[CV] .................. C=100, gamma=0.0001, kernel=rbf, total=   0.0s


[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:    2.1s finished


In [15]:
best_parameters

{'C': 100, 'gamma': 0.01, 'kernel': 'rbf'}

In [16]:
# k-fold cross validation
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator=best_estimator, X=X_train, y=y_train, cv=10, n_jobs=-1,
        scoring='neg_mean_squared_error', verbose=2)
accuracies.mean(), accuracies.std()

[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV] ................................................. , total=   0.0s
[CV] ................................................. , total=   0.0s
[CV]  ................................................................
[CV]  ................................................................
[CV] ................................................. , total=   0.0s
[CV] ................................................. , total=   0.0s
[CV]  ................................................................
[CV]  ................................................................
[CV] ................................................. , total=   0.0s
[CV]  ................................................................
[CV] .

[Parallel(n_jobs=-1)]: Done   3 out of  10 | elapsed:    0.1s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.1s finished


(-0.9435111191272043, 0.0969847010053772)

In [17]:
evaluate(best_estimator, X_test, y_test)

Model Validation
MSE: 1.2170209477451335
R^2: -0.01905803469780354


In [18]:
evalForecast(best_estimator, X_train, y_train, inverse=True, scaler=scaler)

Validasi RMSE: 3.10321
Validasi MAE: 2.54404


In [19]:
joblib.dump(best_estimator, './training/forecast_svr_5aa39533ae1f941be7165ecd_UI.pkl')

['./training/forecast_svr_5aa39533ae1f941be7165ecd_UI.pkl']

## <font color=blue> 3. Mongodb ID: 5a93e8768cbad97881597597 </font>

In [28]:
product = genData('5a93e8768cbad97881597597', conn, c, impute=True, freq='daily')
X_train, y_train, X_test, y_test, dftrain, scaler = splitDataNN(product, percent=0.2)

In [29]:
regressor = SVR(kernel='poly')

# Grid Search
from sklearn.model_selection import GridSearchCV
parameters = [{'C':[1, 5, 10, 100], 'kernel':['linear']},
        {'C':[1, 5, 10, 100], 'kernel':['rbf'], 'gamma':[0.5, 0.1, 0.01, 0.001, 0.0001]}
        ]
grid_search = GridSearchCV(estimator=regressor,
        param_grid=parameters,
        scoring=['neg_mean_squared_error', 'r2', 'explained_variance'],
        cv=5,
        n_jobs=-1,
        refit='r2',
        verbose=2)

In [30]:
grid_search = grid_search.fit(X_train, y_train)
best_accuracy = grid_search.best_score_
best_parameters = grid_search.best_params_
best_estimator = grid_search.best_estimator_

Fitting 5 folds for each of 24 candidates, totalling 120 fits
[CV] C=1, kernel=linear ..............................................
[CV] C=1, kernel=linear ..............................................
[CV] C=1, kernel=linear ..............................................
[CV] C=1, kernel=linear ..............................................
[CV] C=1, kernel=linear ..............................................
[CV] C=5, kernel=linear ..............................................
[CV] C=5, kernel=linear ..............................................
[CV] C=5, kernel=linear ..............................................
[CV] C=10, kernel=linear .............................................
[CV] C=5, kernel=linear ..............................................
[CV] C=10, kernel=linear .............................................
[CV] C=5, kernel=linear ..............................................
[CV] ............................... C=1, kernel=linear, total=   0.0s
[CV] ..........

[CV] C=10, gamma=0.0001, kernel=rbf ..................................
[CV] ..................... C=1, gamma=0.001, kernel=rbf, total=   0.0s
[CV] C=5, gamma=0.01, kernel=rbf .....................................
[CV] ..................... C=100, gamma=0.5, kernel=rbf, total=   0.1s
[CV] ....................... C=5, gamma=0.1, kernel=rbf, total=   0.0s
[CV] C=5, gamma=0.1, kernel=rbf ......................................
[CV] C=10, gamma=0.01, kernel=rbf ....................................
[CV] .................... C=1, gamma=0.0001, kernel=rbf, total=   0.0s
[CV] C=1, gamma=0.001, kernel=rbf ....................................
[CV] ....................... C=1, gamma=0.1, kernel=rbf, total=   0.0s
[CV] C=100, gamma=0.5, kernel=rbf ....................................
[CV] C=1, gamma=0.1, kernel=rbf ......................................
[CV] C=1, gamma=0.0001, kernel=rbf ...................................
[CV] ...................... C=10, gamma=0.5, kernel=rbf, total=   0.1s
[CV] C

[CV] .................. C=100, gamma=0.0001, kernel=rbf, total=   0.0s
[CV] ...................... C=10, gamma=0.5, kernel=rbf, total=   0.0s
[CV] ................... C=100, gamma=0.001, kernel=rbf, total=   0.0s
[CV] ...................... C=10, gamma=0.1, kernel=rbf, total=   0.0s
[CV] ..................... C=100, gamma=0.1, kernel=rbf, total=   0.1s
[CV] C=100, gamma=0.1, kernel=rbf ....................................
[CV] ..................... C=100, gamma=0.1, kernel=rbf, total=   0.0s
[CV] C=100, gamma=0.1, kernel=rbf ....................................
[CV] ..................... C=100, gamma=0.1, kernel=rbf, total=   0.0s


[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:    1.2s finished


In [31]:
best_parameters

{'C': 5, 'gamma': 0.01, 'kernel': 'rbf'}

In [32]:
# k-fold cross validation
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator=best_estimator, X=X_train, y=y_train, cv=10, n_jobs=-1,
        scoring='neg_mean_squared_error', verbose=2)
accuracies.mean(), accuracies.std()

[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV] ................................................. , total=   0.0s
[CV] ................................................. , total=   0.0s
[CV] ................................................. , total=   0.0s
[CV] ................................................. , total=   0.0s
[CV] .

[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.1s finished


(-0.9906324644559131, 0.2409863543186313)

In [33]:
evaluate(best_estimator, X_test, y_test)

Model Validation
MSE: 1.0964962250568069
R^2: -0.025089893752072934


In [34]:
evalForecast(best_estimator, X_train, y_train, inverse=True, scaler=scaler)

Validasi RMSE: 11.41206
Validasi MAE: 9.10734


In [43]:
#joblib.dump(best_estimator, './training/forecast_svr_5a93e8768cbad97881597597_univariate.pkl')

## <font color=blue> 5. Mongodb ID: 5aa2c35e35d6d34b0032a796 </font>

In [35]:
product = genData('5aa2c35e35d6d34b0032a796', conn, c, impute=True, freq='daily')
X_train, y_train, X_test, y_test, dftrain, scaler = splitDataNN(product, percent=0.2)

In [36]:
regressor = SVR(kernel='poly')

# Grid Search
from sklearn.model_selection import GridSearchCV
parameters = [{'C':[1, 5, 10, 100], 'kernel':['linear']},
        {'C':[1, 5, 10, 100], 'kernel':['rbf'], 'gamma':[0.5, 0.1, 0.01, 0.001, 0.0001]}
        ]
grid_search = GridSearchCV(estimator=regressor,
        param_grid=parameters,
        scoring=['neg_mean_squared_error', 'r2', 'explained_variance'],
        cv=5,
        n_jobs=-1,
        refit='r2',
        verbose=2)

In [37]:
grid_search = grid_search.fit(X_train, y_train)
best_accuracy = grid_search.best_score_
best_parameters = grid_search.best_params_
best_estimator = grid_search.best_estimator_

Fitting 5 folds for each of 24 candidates, totalling 120 fits
[CV] C=1, kernel=linear ..............................................
[CV] C=1, kernel=linear ..............................................
[CV] C=1, kernel=linear ..............................................
[CV] C=1, kernel=linear ..............................................
[CV] C=1, kernel=linear ..............................................
[CV] C=5, kernel=linear ..............................................
[CV] C=5, kernel=linear ..............................................
[CV] ............................... C=1, kernel=linear, total=   0.0s
[CV] C=5, kernel=linear ..............................................
[CV] C=5, kernel=linear ..............................................
[CV] ....................... C=1, gamma=0.5, kernel=rbf, total=   0.0s
[CV] ............................... C=1, kernel=linear, total=   0.0s
[CV] C=5, kernel=linear ..............................................
[CV] ..........

[CV] C=5, gamma=0.01, kernel=rbf .....................................
[CV] ..................... C=100, gamma=0.1, kernel=rbf, total=   0.0s
[CV] C=100, gamma=0.1, kernel=rbf ....................................
[CV] ...................... C=1, gamma=0.01, kernel=rbf, total=   0.0s
[CV] ..................... C=10, gamma=0.01, kernel=rbf, total=   0.0s
[CV] C=1, gamma=0.01, kernel=rbf .....................................
[CV] ...................... C=5, gamma=0.01, kernel=rbf, total=   0.0s
[CV] C=10, gamma=0.01, kernel=rbf ....................................
[CV] ..................... C=100, gamma=0.1, kernel=rbf, total=   0.0s
[CV] C=100, gamma=0.1, kernel=rbf ....................................
[CV] ...................... C=1, gamma=0.01, kernel=rbf, total=   0.0s
[CV] C=5, gamma=0.01, kernel=rbf .....................................
[CV] ..................... C=10, gamma=0.01, kernel=rbf, total=   0.0s
[CV] C=1, gamma=0.01, kernel=rbf .....................................
[CV] .

[CV] .................. C=100, gamma=0.0001, kernel=rbf, total=   0.0s
[CV] C=5, gamma=0.5, kernel=rbf ......................................
[CV] C=100, gamma=0.0001, kernel=rbf .................................
[CV] ....................... C=5, gamma=0.5, kernel=rbf, total=   0.0s
[CV] .................. C=100, gamma=0.0001, kernel=rbf, total=   0.0s
[CV] C=100, gamma=0.0001, kernel=rbf .................................
[CV] .................. C=100, gamma=0.0001, kernel=rbf, total=   0.0s
[CV] C=100, gamma=0.0001, kernel=rbf .................................
[CV] .................. C=100, gamma=0.0001, kernel=rbf, total=   0.0s


[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:    0.2s finished


In [38]:
best_parameters

{'C': 1, 'kernel': 'linear'}

In [39]:
# k-fold cross validation
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator=best_estimator, X=X_train, y=y_train, cv=10, n_jobs=-1,
        scoring='neg_mean_squared_error', verbose=2)
accuracies.mean(), accuracies.std()

[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV] ................................................. , total=   0.0s
[CV] ................................................. , total=   0.0s
[CV]  ................................................................
[CV] ................................................. , total=   0.0s
[CV]  ................................................................
[CV] ................................................. , total=   0.0s
[CV] ................................................. , total=   0.0s
[CV]  ................................................................
[CV]  

[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.0s finished


(-0.8372595422238925, 1.0131818816484663)

In [40]:
evaluate(best_estimator, X_test, y_test)

Model Validation
MSE: 1.6452804904425775
R^2: -0.00579370059787343


In [41]:
evalForecast(best_estimator, X_train, y_train, inverse=True, scaler=scaler)

Validasi RMSE: 0.43130
Validasi MAE: 0.09309


In [63]:
#joblib.dump(best_estimator, './training/forecast_svr_5aa2c35e35d6d34b0032a796_univariate.pkl')

## <font color=blue> 7. Mongodb ID: 5a9347b98cbad97074cb1890 </font>

In [42]:
product = genData('5a9347b98cbad97074cb1890', conn, c, impute=True, freq='daily')
X_train, y_train, X_test, y_test, dftrain, scaler = splitDataNN(product, percent=0.2)

In [43]:
from sklearn.svm import SVR
regressor = SVR(kernel='poly')

In [44]:
# Grid Search
from sklearn.model_selection import GridSearchCV
parameters = [{'C':[1, 5, 10, 100], 'kernel':['linear']},
        {'C':[1, 5, 10, 100], 'kernel':['rbf'], 'gamma':[0.5, 0.1, 0.01, 0.001, 0.0001]}
        ]
grid_search = GridSearchCV(estimator=regressor,
        param_grid=parameters,
        scoring=['neg_mean_squared_error', 'r2', 'explained_variance'],
        cv=5,
        n_jobs=-1,
        refit='r2',
        verbose=2)

In [45]:
grid_search = grid_search.fit(X_train, y_train)
best_accuracy = grid_search.best_score_
best_parameters = grid_search.best_params_
best_estimator = grid_search.best_estimator_

Fitting 5 folds for each of 24 candidates, totalling 120 fits
[CV] C=1, kernel=linear ..............................................
[CV] C=1, kernel=linear ..............................................
[CV] C=1, kernel=linear ..............................................
[CV] C=1, kernel=linear ..............................................
[CV] C=1, kernel=linear ..............................................
[CV] C=5, kernel=linear ..............................................
[CV] ............................... C=1, kernel=linear, total=   0.0s
[CV] ............................... C=1, kernel=linear, total=   0.0s
[CV] C=5, kernel=linear ..............................................
[CV] C=10, kernel=linear .............................................
[CV] ............................... C=1, kernel=linear, total=   0.0s
[CV] ............................... C=1, kernel=linear, total=   0.0s
[CV] C=5, kernel=linear ..............................................
[CV] C=5, kerne

[CV] ..................... C=10, gamma=0.01, kernel=rbf, total=   0.0s
[CV] C=5, gamma=0.0001, kernel=rbf ...................................
[CV] C=10, gamma=0.01, kernel=rbf ....................................
[CV] C=1, gamma=0.01, kernel=rbf .....................................
[CV] C=5, gamma=0.5, kernel=rbf ......................................
[CV] ..................... C=100, gamma=0.1, kernel=rbf, total=   0.0s
[CV] C=100, gamma=0.1, kernel=rbf ....................................
[CV] ...................... C=1, gamma=0.01, kernel=rbf, total=   0.0s
[CV] .................. C=100, gamma=0.0001, kernel=rbf, total=   0.0s
[CV] ..................... C=10, gamma=0.01, kernel=rbf, total=   0.0s
[CV] C=1, gamma=0.01, kernel=rbf .....................................
[CV] C=10, gamma=0.01, kernel=rbf ....................................
[CV] ....................... C=5, gamma=0.5, kernel=rbf, total=   0.0s
[CV] .................... C=5, gamma=0.0001, kernel=rbf, total=   0.0s
[CV] C

[CV] C=5, gamma=0.01, kernel=rbf .....................................
[CV] C=100, gamma=0.001, kernel=rbf ..................................
[CV] ................... C=10, gamma=0.0001, kernel=rbf, total=   0.0s
[CV] C=100, gamma=0.5, kernel=rbf ....................................
[CV] ...................... C=5, gamma=0.01, kernel=rbf, total=   0.0s
[CV] ..................... C=100, gamma=0.5, kernel=rbf, total=   0.0s
[CV] ................... C=100, gamma=0.001, kernel=rbf, total=   0.0s
[CV] C=100, gamma=0.001, kernel=rbf ..................................
[CV] ................... C=100, gamma=0.001, kernel=rbf, total=   0.0s


[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:    0.2s finished


In [46]:
best_parameters

{'C': 100, 'gamma': 0.1, 'kernel': 'rbf'}

In [47]:
# k-fold cross validation
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator=best_estimator, X=X_train, y=y_train, cv=10, n_jobs=-1,
        scoring='neg_mean_squared_error', verbose=2)
accuracies.mean(), accuracies.std()

[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV]  ................................................................
[CV] ................................................. , total=   0.0s
[CV]  ................................................................
[CV]  ................................................................
[CV] ................................................. , total=   0.0s
[CV]  ................................................................
[CV] ................................................. , total=   0.0s
[CV] ................................................. , total=   0.0s
[CV]  ................................................................
[CV]  ................................................................
[CV] .

[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.0s finished


(-1.1071626625335298, 0.385806103303906)

In [48]:
evaluate(best_estimator, X_train, y_train)

Model Validation
MSE: 1.105426282821663
R^2: -0.1130162963541077


In [49]:
evalForecast(best_estimator, X_train, y_train, inverse=True, scaler=scaler)

Validasi RMSE: 0.42199
Validasi MAE: 0.20136


In [ ]:
#joblib.dump(best_estimator, './training/forecast_svr_5a9347b98cbad97074cb1890_univariate.pkl')

## performansi pada train set

In [33]:
product = genData('5aa2c35e35d6d34b0032a796', conn, c, impute=True, freq='daily')
X_train, y_train, X_test, y_test, dftrain, scaler = splitDataNN(product, percent=0.2)

In [34]:
X_train = X_train[:,0]
X_test = X_test[:, 0]
X_train = X_train.reshape((-1, 1))
X_test = X_test.reshape((-1, 1))

In [37]:
from sklearn.svm import SVR
best_estimator = SVR(C=1, kernel='linear')
best_estimator.fit(X_train, y_train)

SVR(C=1, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='linear', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [38]:
evaluate(best_estimator, X_train, y_train)

Model Validation
MSE: 0.8372595422238925
R^2: -3.183356889202571e-08


In [39]:
y_pred = best_estimator.predict(X_train)
np.sqrt(mean_squared_error(y_pred=y_pred, y_true=y_train.reshape(-1,1)))

0.915018875337494

In [40]:
mean_absolute_error(y_pred=y_pred, y_true=y_train.reshape(-1,1))

0.19748380148557873